In [4]:
%matplotlib inline

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import os

In [6]:
eng_df = pd.read_csv("transfers_output/england_joined.csv")
italy_df = pd.read_csv("transfers_output/italy_joined.csv")
spain_df = pd.read_csv("transfers_output/spain_joined.csv")

In [7]:
spain_df = spain_df.fillna(0)
italy_df= italy_df.fillna(0)
eng_df = eng_df.fillna(0)

In [8]:
list_dfs = [eng_df,spain_df,italy_df]

In [9]:
italy_df.head(80)

,Unnamed: 0,Year,Teams,Ranking,avg player value (EU),avg player wage,Transfer Spend (Euros),MA
0,0,2019,Juventus,1.0,27679000.0,130840.0,201.150,194.28
1,1,2019,Inter Milan,2.0,19418125.0,66792.0,171.284,127.64
2,2,2019,SS Lazio,3.0,10884848.0,38273.0,41.078,36.29
3,3,2019,Atalanta,4.0,6632200.0,20880.0,51.130,49.78
4,4,2019,AS Roma,5.0,12983653.0,53769.0,102.690,110.81
...,...,...,...,...,...,...,...,...
75,75,2016,Genoa,16.0,3837413.0,34207.0,24.480,15.40
76,76,2016,FC Crotone,17.0,556041.0,5583.0,5.355,0.00
77,77,2016,FC Empoli,18.0,1839600.0,18440.0,6.300,0.00
78,78,2016,Palermo,19.0,2221481.0,21296.0,7.630,12.13


In [10]:
def plot(list_df,column1,column2):
    leagues = ["Premier league","Liga", "Serie A"]
    count=1
    plt.figure(figsize=(20,8))
    plt.suptitle(f"{column1} vs {column2}",fontsize=20)
    for i,df in enumerate(list_df):
        plt.subplot(1,3,count)
        for j,year in enumerate(sorted(list(df["Year"].unique()))):
            if df[df["Year"]==year][column2].all() == 0:
                continue
            colors = ['blue','red','green','gold','magenta']
            x = df[df["Year"]==year][column1]
            y = df[df["Year"]==year][column2]
            plt.scatter(x,y,label=year,c=colors[j],s=30)
        plt.legend(loc="best")
        plt.xticks(np.arange(round(min(df[column1])),round(max(df[column1])+1)))
        plt.ylim(min(df[column2])-0.75,max(df[column2])+0.5)
        avg = round(df[column2].mean(),2)    
        plt.title(leagues[i], fontsize=14,color="b",pad=15)
        plt.axhline(avg)
        plt.text(min(df[column1])-1,avg,f"{avg}",horizontalalignment='right',color="r")
        plt.tight_layout()
        count+=1
        #r = stats.pearsonr(df[column1],df[column2])[0]
        #plt.annotate(f"r-squared : {round(r**2,2)}",(max(df[column1]-5),max(df[column2])+0.5),fontsize=14, color= "r")


In [11]:
def r_square(list_dfs, col1, col2):
    leagues = ["Premier league","Liga", "Serie A"]
    for league, df in zip(leagues, list_dfs):
        name = league
        r = stats.pearsonr(df[col1],df[col2])[0]
        r_2 = round(r**2,3)
        print(f"When considering {col1} vs {col2}, The R-Square of {name} is {r_2}")

In [114]:
r_square(list_dfs,"Ranking","MA")

When considering Ranking vs MA, The R-Square of Premier league is 0.442
When considering Ranking vs MA, The R-Square of Liga is 0.352
When considering Ranking vs MA, The R-Square of Serie A is 0.326


In [126]:
r_square(list_dfs,"Ranking","avg player wage")

When considering Ranking vs avg player wage, The R-Square of Premier league is 0.269
When considering Ranking vs avg player wage, The R-Square of Liga is 0.191
When considering Ranking vs avg player wage, The R-Square of Serie A is 0.225


In [128]:
r_square(list_dfs,"Ranking","Transfer Spend (Euros)")

When considering Ranking vs Transfer Spend (Euros), The R-Square of Premier league is 0.212
When considering Ranking vs Transfer Spend (Euros), The R-Square of Liga is 0.255
When considering Ranking vs Transfer Spend (Euros), The R-Square of Serie A is 0.303


In [129]:
r_square(list_dfs,"Ranking","avg player value (EU)")

When considering Ranking vs avg player value (EU), The R-Square of Premier league is 0.34
When considering Ranking vs avg player value (EU), The R-Square of Liga is 0.205
When considering Ranking vs avg player value (EU), The R-Square of Serie A is 0.219


In [2]:
def compare_average(df,column):
    years = eng_df["Year"].unique().tolist()
    
    league_average = []
    
    for year in years:
        filtered_df = df[df["Year"]==year]
        average = round(filtered_df[column].mean(),2)
        league_average.append(average)
    result = {years[i]: league_average[i] for i in range(len(years))} 

    league_df = pd.DataFrame.from_dict(result, orient = "index").reset_index()
    league_df = league_df.rename(columns={0:f"League Avg: {column}","index":"Year"})

    top4_average = []
    
    for year in years:
        filtered_df_2 = df[(df["Year"]==year) & (df["Ranking"]<5)]
        average = round(filtered_df_2[column].mean(),2)
        top4_average.append(average)

    result_2 = {years[i]: top4_average[i] for i in range(len(years))} 
    top4_df = pd.DataFrame.from_dict(result_2, orient = "index").reset_index()
    top4_df = top4_df.rename(columns={0:f"Top 4 Avg: {column}","index":"Year"})
    combined_df = pd.merge(league_df,top4_df,on=["Year"])
    return combined_df

In [203]:
eng_MA = compare_average(eng_df,"avg player wage")
eng_MA["Difference"] = eng_MA["Top 4 Avg: avg player wage"]-eng_MA["League Avg: avg player wage"]
eng_MA

,Year,League Avg: avg player wage,Top 4 Avg: avg player wage,Difference
0,2019,50346.15,101666.75,51320.60
1,2018,54770.55,88818.25,34047.70
2,2017,29464.45,56860.00,27395.55
3,2016,47307.45,83082.00,35774.55
4,2015,0.00,0.00,0.00


In [13]:
italy_comp = compare_average(italy_df,"avg player wage")
italy_comp["Difference"] = italy_comp["Top 4 Avg: avg player wage"]-italy_comp["League Avg: avg player wage"]
italy_comp

,Year,League Avg: avg player wage,Top 4 Avg: avg player wage,Difference
0,2019,32934.10,51357.0,18422.90
1,2018,33665.62,52559.2,18893.58
2,2017,24481.90,49557.8,25075.90
3,2016,37014.29,54919.8,17905.51
4,2015,0.00,0.0,0.00


In [12]:
eng_MA = compare_average(eng_df,"MA")
eng_MA["Difference"] = eng_MA["Top 4 Avg: MA"]-eng_MA["League Avg: MA"]
eng_MA

,Year,League Avg: MA,Top 4 Avg: MA,Difference
0,2019,66.33,145.41,79.08
1,2018,64.22,139.06,74.84
2,2017,69.54,145.23,75.69
3,2016,53.88,108.59,54.71
4,2015,43.00,64.90,21.90
